<a href="https://colab.research.google.com/github/CalderLeong/CalderLeong/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load data
train_data = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["type", "msg"])
test_data = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["type", "msg"])

# Consistent label mapping
label_map = {"ham": 0, "spam": 1}
train_data["type"] = train_data["type"].map(label_map)
test_data["type"] = test_data["type"].map(label_map)

# Extract labels **after mapping**
train_labels = train_data["type"].values
test_labels = test_data["type"].values
train_messages = train_data["msg"].values
test_messages = test_data["msg"].values

# Text vectorization
max_features = 1000
max_len = 100

vectorize_layer = keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=max_len
)
vectorize_layer.adapt(train_messages)

# Build the model (with optional dense layer for improvement)
model = keras.Sequential([
    vectorize_layer,
    keras.layers.Embedding(max_features + 1, 32),
    keras.layers.Dropout(0.2),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),  # added for better performance
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

# Train model
epochs = 20
history = model.fit(
    train_messages,
    train_labels,
    epochs=epochs,
    validation_data=(test_messages, test_labels),
    verbose=2
)


In [ ]:
def predict_message(pred_text, threshold=0.25):  # changed threshold to 0.25
    pred_text = tf.convert_to_tensor([pred_text])
    pred = model.predict(pred_text)
    return [float(pred[0][0]), "ham" if pred[0][0] < threshold else "spam"]
def test_predictions_with_threshold(threshold=0.25):
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too"
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg, threshold=threshold)
        print(f"Message: {msg}")
        print(f"Expected: {ans} | Predicted: {prediction[1]} ({prediction[0]:.4f})")
        print("-" * 50)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("✅ You passed the challenge. Great job!")
    else:
        print("❌ You haven't passed yet. Keep trying.")

# Run this
test_predictions_with_threshold()


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
